In [115]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import sklearn
import scipy
from sklearn.preprocessing import OrdinalEncoder

In [116]:
df = pd.read_csv('./WA_Marketing-Campaign.csv')
df['Promotion'].unique() # Three different marketing promotions
df.head(5)
df.describe()

,MarketID,LocationID,AgeOfStore,Promotion,week,SalesInThousands
count,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000
mean,5.715328,479.656934,8.503650,2.029197,2.500000,53.466204
std,2.877001,287.973679,6.638345,0.810729,1.119055,16.755216
min,1.000000,1.000000,1.000000,1.000000,1.000000,17.340000
25%,3.000000,216.000000,4.000000,1.000000,1.750000,42.545000
50%,6.000000,504.000000,7.000000,2.000000,2.500000,50.200000
75%,8.000000,708.000000,12.000000,3.000000,3.250000,60.477500
max,10.000000,920.000000,28.000000,3.000000,4.000000,99.650000


In [117]:
df.head(10)

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
0,1,Medium,1,4,3,1,33.73
1,1,Medium,1,4,3,2,35.67
2,1,Medium,1,4,3,3,29.03
3,1,Medium,1,4,3,4,39.25
4,1,Medium,2,5,2,1,27.81
5,1,Medium,2,5,2,2,34.67
6,1,Medium,2,5,2,3,27.98
7,1,Medium,2,5,2,4,27.72
8,1,Medium,3,12,1,1,44.54
9,1,Medium,3,12,1,2,37.94


In [118]:
marketsizes = df['MarketSize'].unique()
marketsizes[0] = 'Small'
marketsizes[1] = 'Medium'
marketsizes
sizeEncoder = OrdinalEncoder(categories=[marketsizes])
df['MarketSize'] = sizeEncoder.fit_transform(df[['MarketSize']])

promotion1 = df[df['Promotion'] == 1]
promotion2 = df[df['Promotion'] == 2]
promotion3 = df[df['Promotion'] == 3]
promotion1.describe()

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
count,172.000000,172.000000,172.000000,172.00000,172.0,172.000000,172.000000
mean,5.790698,1.209302,488.465116,8.27907,1.0,2.500000,58.099012
std,2.993624,0.632757,299.352389,6.63616,0.0,1.121298,16.553782
min,1.000000,0.000000,3.000000,1.00000,1.0,1.000000,30.810000
25%,3.000000,1.000000,218.000000,3.00000,1.0,1.750000,46.352500
50%,6.000000,1.000000,512.000000,6.00000,1.0,2.500000,55.385000
75%,8.000000,2.000000,711.000000,12.00000,1.0,3.250000,63.617500
max,10.000000,2.000000,919.000000,27.00000,1.0,4.000000,99.650000


In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv("config.env")
username = os.getenv("username")
password = os.getenv("password")

con = "mysql+mysqlconnector://{user}:{pwd}@{host}/{db}"
con = con.format(user = username, pwd = password, host = 'localhost', db = 'fast_food_marketing')
engine = sqlalchemy.create_engine(con)
connection = engine.connect()
df.to_sql('marketing_data', connection, if_exists='append')

548

In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    promotion1 = pd.DataFrame(conn.execute(text("SELECT * FROM marketing_data WHERE Promotion = 1")))
    promotion2 = pd.DataFrame(conn.execute(text("SELECT * FROM marketing_data WHERE Promotion = 2")))
    promotion3 = pd.DataFrame(conn.execute(text("SELECT * FROM marketing_data WHERE Promotion = 3")))

print(promotion1)

     index  MarketID  MarketSize  LocationID  AgeOfStore  Promotion  week  \
0        8         1         1.0           3          12          1     1   
1        9         1         1.0           3          12          1     2   
2       10         1         1.0           3          12          1     3   
3       11         1         1.0           3          12          1     4   
4       24         1         1.0           7          15          1     1   
..     ...       ...         ...         ...         ...        ...   ...   
167    539        10         2.0         918           6          1     4   
168    540        10         2.0         919           2          1     1   
169    541        10         2.0         919           2          1     2   
170    542        10         2.0         919           2          1     3   
171    543        10         2.0         919           2          1     4   

     SalesInThousands  
0               44.54  
1               37.94  
2  

In [ ]:
# ANOVA for multiple groups
# requires equal variances, so we use Levene's test to determine whether the population variances are equal
from scipy.stats import levene
p_val_lev = levene(promotion1['SalesInThousands'].to_numpy(), promotion2['SalesInThousands'].to_numpy(), promotion3['SalesInThousands'].to_numpy())
p_val_lev
# p-value is 0.28, null hypothesis in Levene's test is not rejected
# Therefore, we can conclude that the variance of the sales in thousands is equal for all three campaigns. 

In [ ]:
# Because the variances are the same, we can use Fisher's ANOVA,
# as this test requires the assumption of equal variances.
from scipy.stats import f_oneway
anova = f_oneway(promotion1['SalesInThousands'].to_numpy(), promotion2['SalesInThousands'].to_numpy(), promotion3['SalesInThousands'].to_numpy())
anova
# At a significance level of 5%, there is enough evidence to conclude that the means differ. 

In [ ]:
# Use Tukey's test
# need to check assumptions that sample sizes are equal
samplesize1 = promotion1.shape
samplesize2 = promotion2.shape
samplesize3 = promotion3.shape
print(f"{samplesize1}, {samplesize2}, {samplesize3}")

# unequal sample sizes - need to use Tukey-Kramer Method
means = [np.mean(promotion1['SalesInThousands'].to_numpy()), np.mean(promotion2['SalesInThousands'].to_numpy()), np.mean(promotion3['SalesInThousands'].to_numpy())]
print(f"{means}")
# q-value


In [ ]:
np.unique(np.array(df['LocationID']))

In [ ]:
df['LocationID'].describe()
df['LocationID'].shape